In [12]:
import numpy as np
import scipy.sparse as sparse
import warnings
from numpy.linalg import eigvals
from scipy.sparse.linalg import expm, eigs
from qiskit.opflow import PrimitiveOp, PauliTrotterEvolution
from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit import Aer, execute, QuantumCircuit

# ignore sparse efficiency warnings
warnings.simplefilter('ignore', sparse.SparseEfficiencyWarning)

# All matrix functions operate with Scipy spare csc as input/return type

def char_to_pauli(char):
    """Returns the Pauli matrix for a given character in I, X, Y, Z"""
    char = char.upper() # convert lower to upper case
    if char == 'I':
        return np.matrix([[1,0],[0,1]])
    if char == 'X':
        return np.matrix([[0,1],[1,0]])
    if char == 'Y':
        return np.matrix([[0,-1j],[1j,0]])
    if char == 'Z':
        return np.matrix([[1,0],[0,-1]])
    raise Exception("Character '" + char + "' does not correspond to Pauli matrix")

def string_to_operator(string):
    """Calculates the operator (Kronecker product) from a Pauli string"""
    op = sparse.csc_matrix(1, dtype=np.complex128)
    for c in string:
        op = sparse.kron(op, char_to_pauli(c))
    return op

def pauli_unitary(string, coefficient):
    """Calculates the unitary matrix exp(-icH) for a given Pauli string H with coefficient c"""
    return expm(-1j * coefficient * string_to_operator(string))

def pauli_circuit(string, coefficient):
    """Quantum circuit for the given Pauli string with coefficient"""
    op = PrimitiveOp(SparsePauliOp(string, coefficient))
    unitary = PauliTrotterEvolution().convert(op.exp_i())
    return unitary.to_circuit().decompose()

def circuit_unitary(circuit, simulate = False):
    """Return unitary calculated from Quantum circuit
       if parameter simulate is True: simulate with Aer unitary backend
       default: obtain unitary from qiskit.Operator class
    """
    if simulate:
        backend = Aer.get_backend('unitary_simulator')
        job = execute(circuit, backend)
        result = job.result()
        u = result.get_unitary(circuit)
    else:
        u = Operator(circuit).data
    return sparse.csc_matrix(u, dtype=np.complex128)

def error(a, b, sparse = True):
    """Error between two equal sized unitaries a, b (sparse csc) defined
    as largest magnitude eigenvalue of a-b
    """
    if sparse:
        ev = eigs(a-b, 1, which='LM', return_eigenvectors=False)
        return np.abs(ev[0])
    else:
        return np.max(np.abs(eigvals(a.toarray()-b.toarray())))

class PauliHamiltonian:
    """Class for Hamiltonian built from Pauli strings
    Attributes:
        operators: list of Pauli strings, in order of coefficient size
        coefficients: dictionary {'Pauli String': coefficient}
        qubits: number of qbits 
    """
    operators = []
    coefficients = {}
    qubits = 0
    
    __matrix = None
    __unitary = None
    
    def matrix(self):
        """Hamiltonian in matrix form"""
        if self.__matrix is None: # matrix form has not been calculated yet
            n = self.dim()
            self.__matrix = sparse.csc_matrix((n, n), dtype=np.complex128)
            for o in self.operators:
                self.__matrix += self.coefficients[o] * string_to_operator(o)
        return self.__matrix
    
    def unitary(self):
        """exp(-iH) in matrix form"""
        if self.__unitary is None: # unitary has not been calculated yet
            self.__unitary = expm(-1j * self.matrix())
        return self.__unitary
    
    def dim(self):
        """Matrix dimension 2^n with n qubits"""
        return 2**self.qubits
    
    def approx_from_unitaries(self, ops = None):
        """Unitary that approximates the full Hamiltonian evolution exp(-iH)
           calculated as matrix product of the single Pauli unitaries for each string:
           U = U1 * U2 ... * Un where Ui are in order as given by ops
           ops: list of Pauli Strings, subset of self.operators
           if ops is None: ops = self.operators
        """
        if ops is None:
            ops = self.operators
        u = sparse.eye(self.dim(), format='csc') # initialize with identity matrix
        for o in ops:
            u = u.dot(pauli_unitary(o, self.coefficients[o]))
        return u
            
    def approx_from_circuit(self, ops = None, **options):
        """Unitary that approximates the full Hamiltonian evolution exp(-iH)
           calculated as a circuit combination of the single Pauli circuits for each string.
           ops: list of Pauli Strings, subset of self.operators
           if ops is None: ops = self.operators
           simulate = True can be given as additional option to simulate with Aer
           (default is to retrieve unitary from Operator.data)
        """
        simulate = False
        if 'simulate' in options:
            simulate = options['simulate']
        if ops is None:
            ops = self.operators    
        if len(ops) < 1:
            return 0
        qc = QuantumCircuit(self.qubits)
        global_phase = 0
        # go through ops in reverse order to built circuit with rightmost unitary first
        for i in range(len(ops)-1,-1,-1):
            if ops[i] == 'I'*self.qubits:
                global_phase += self.coefficients[ops[i]]
            else:
                qc.append(pauli_circuit(ops[i], self.coefficients[ops[i]]),range(self.qubits))
        return np.exp(-1j*global_phase) * circuit_unitary(qc, simulate)
    
            
def hamiltonian_from_file(filename):
    """Read Hamiltonian from file and return PauliHamiltonian object.
    File format: 'coefficient Pauli-string' per line, separated by space"""
    try:
        file = open(filename) # read file
    except:
        print("An exception occurred trying to read the file: " + filename) 
    lines = file.read().split('\n')
    h = PauliHamiltonian() # create object to return
    for l in lines:
        if len(l): # exclude empty lines
            [c,o] = l.split() # split in coefficient c and operator string o
            if h.qubits:
                if len(o) != h.qubits: # Make sure all operators have the same dimension
                    raise Exception("Operator string '" + o + "' does not match dimension (" + h.qubits + ")")
            else:
                h.qubits = len(o) # set no. qubits to length of operator string
            h.operators.append(o) # add operator to list of operators of object h
            h.coefficients[o] = float(c) # add to the dictionary of coefficients for h
    return h


h = hamiltonian_from_file("hamiltonian_ordered.txt")

In [2]:
# max error of all Pauli unitaries:
m = 0
for o in h.operators:
    c = h.coefficients[o]
    e = error(pauli_unitary(o,c), circuit_unitary(pauli_circuit(o,c)))
    if e > m:
        m = e
print(m)

8.881648663456881e-16


In [2]:
%%time
print("Calculating full unitary")
u = h.unitary()

Calculating full unitary
CPU times: user 13.2 s, sys: 1.54 s, total: 14.8 s
Wall time: 14.7 s


In [5]:
%%time
print("Calculating approximating unitary from unitaries")
u1 = h.approx_from_unitaries()

Calculating approximating unitary from unitaries
CPU times: user 18min 40s, sys: 3.41 s, total: 18min 44s
Wall time: 18min 44s


In [13]:
%%time
print("Calculating approximating unitary from circuit")
u2 = h.approx_from_circuit()

Calculating approximating unitary from circuit
CPU times: user 1min 3s, sys: 24.2 ms, total: 1min 3s
Wall time: 1min 3s


In [14]:
%%time
print("Calculating approximating unitary from circuit (using simulator)")
u3 = h.approx_from_circuit(simulate=True)

Calculating approximating unitary from circuit (using simulator)
CPU times: user 32.3 s, sys: 8.24 s, total: 40.5 s
Wall time: 8.22 s


In [6]:
%%time
error(u,u1)

CPU times: user 246 ms, sys: 188 ms, total: 434 ms
Wall time: 75.5 ms


0.07406384276827156

In [7]:
%%time
error(u,u1,False) # no sparse

CPU times: user 3.61 s, sys: 1.7 s, total: 5.3 s
Wall time: 884 ms


0.07406384276827152

In [15]:
%%time
error(u,u2)

CPU times: user 385 ms, sys: 332 ms, total: 718 ms
Wall time: 123 ms


0.07406384276827217

In [18]:
%%time
error(u,u3)

CPU times: user 2.27 s, sys: 1.71 s, total: 3.98 s
Wall time: 695 ms


0.07406384276826321

In [16]:
%%time
error(u,u2,False) # no sparse

CPU times: user 3.93 s, sys: 1.73 s, total: 5.66 s
Wall time: 944 ms


0.07406384276827183

In [17]:
%%time
error(u,u3,False) # no sparse

CPU times: user 4.43 s, sys: 1.71 s, total: 6.14 s
Wall time: 1.02 s


0.07406384276826339

In [19]:
%%time
# Same for the Hamiltonian chopped at 191:
print("Calculating full unitary")
u = h.unitary()
ops = h.operators[:191]
print("Calculating approximating unitary from circuit")
u2 = h.approx_from_circuit(ops)
print("Calculating approximating unitary from circuit (using simulator)")
u3 = h.approx_from_circuit(ops, simulate=True)
print("Error circuit: ", error(u,u2))
print("Error with simulator: ", error(u,u3))

Calculating full unitary
Calculating approximating unitary from circuit
Calculating approximating unitary from circuit (using simulator)
Error circuit:  0.0984674492173192
Error with simulator:  0.09846744921731607
CPU times: user 1min 15s, sys: 5.09 s, total: 1min 20s
Wall time: 60 s
